In [5]:
!pip install "flaml[automl]"
!pip install rapidfuzz


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Zhihao\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Zhihao\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os

In [8]:
from nq_agents import WorkflowAutogen
from nq_agents import data_extraction

# Initialize the multi-agent system
my_agents = WorkflowAutogen()

# Read input examples
GOLD_FILE = 'data/two-dev.jsonl'
num_examples = data_extraction.count_lines(GOLD_FILE)

input_examples = data_extraction.read_first_lines(GOLD_FILE, num_examples)

# Make predictions. side effect: write to file
save_path = GOLD_FILE.replace('.jsonl', '_predictions.jsonl')
predictions = my_agents.predict_batch(examples=input_examples, save_path=save_path, verbose=True)



Example 1/2
Generated subqueries: ['Can a UCC be filed on an individual', 'What is the Uniform Commercial Code', 'Who can file a UCC against an individual', 'What are the consequences of filing a UCC against an individual', 'Can a UCC be used to collect debts from an individual', 'How does a UCC differ from other types of court actions against an individual', 'Are there any exemptions or exceptions to filing a UCC against an individual', 'What are the benefits and drawbacks of filing a UCC against an individual', "Can a UCC be used to freeze an individual's assets", 'How can I determine if a UCC has been filed on me']
subqueries ['Can a UCC be filed on an individual', 'What is the Uniform Commercial Code', 'Who can file a UCC against an individual', 'What are the consequences of filing a UCC against an individual', 'Can a UCC be used to collect debts from an individual', 'How does a UCC differ from other types of court actions against an individual', 'Are there any exemptions or excep

TypeError: 'NoneType' object is not iterable

### Evaluate


In [ ]:
from nq_agents import evaluation

# Evaluate predictions
GOLD_FILE = 'data/two-dev.jsonl'
PREDICTIONS_FILE = 'data/two-dev_predictions.jsonl' # generated by multi_agent.py

result = evaluation.evaluate_predictions(GOLD_FILE, PREDICTIONS_FILE)

In [ ]:
result

In [ ]:
a = {1:1, 2:2}
b = {1:1, 2:2}
a == b



In [ ]:
a.keys().symmetric_difference(b.keys())

In [ ]:
type(a.keys())